In [3]:
!pip install boto3==1.9.188

In [4]:
from sagemaker.pytorch import PyTorch
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)

sagemaker_session = sagemaker.Session()


In [5]:

bucket=sagemaker_session.default_bucket() 

prefix = "pytorch_load_test"

pretrained_model_path = "s3://pytorch/models/resnet18-5c106cde.pth"

In [6]:
model_path = "s3://{}/{}/source.tar.gz".format(bucket, prefix)

In [7]:
!aws s3 cp $pretrained_model_path .
!tar -czvf source.tar.gz *.pth 

!aws s3 cp  source.tar.gz  $model_path

download: s3://pytorch/models/resnet18-5c106cde.pth to ./resnet18-5c106cde.pth
a resnet18-5c106cde.pth
upload: ./source.tar.gz to s3://sagemaker-us-east-2-324346001917/pytorch_load_test/source.tar.gz


In [ ]:
pytorch_model = sagemaker.pytorch.model.PyTorchModel(model_data=model_path, role=role, 
                             entry_point='./src/sm_inference.py')

predictor = pytorch_model.deploy(instance_type='ml.p3.2xlarge', initial_instance_count=1)

-----------------------------------------------